In [1]:
import torch
import numpy as np
import sys

sys.path.append("monotone_op_net")
import train
import splitting as sp
import mon

%matplotlib inline

In [ ]:
# MNIST Dataloaders
trainLoader, testLoader = train.mnist_loaders(train_batch_size=128, test_batch_size=500)
def cuda(tensor):
    if 0:
        return tensor.cuda()
    else:
        return tensor

<h2> Lipschitz constant of FC monDEQ model

In [3]:
# Instantiate and load FC monDEQ model
m = 20
h = 40
model = train.SingleFcNet(sp.MONPeacemanRachford,
                        in_dim=28**2,
                        out_dim=h,
                        alpha=1.0,
                        max_iter=300,
                        tol=1e-5,
                        m=m)

model.load_state_dict(torch.load("models/mon_h40_m20.0.pt", map_location=torch.device('cpu')))
model.eval()

with torch.no_grad():
    total_cnt = 0
    correct = 0
    for batch in testLoader:
        data, target = cuda(batch[0]), cuda(batch[1])
        preds = model(data)
        target = target.numpy()
        preds = preds.numpy()
        correct += np.sum(np.equal(np.argmax(preds, axis=1), target).astype(int))
        total_cnt += len(target)
print("monDEQ test error = %.3f" % (1-correct / total_cnt))

monDEQ test error = 0.044


In [5]:
# Compute Lipschitz Constant of monDEQ as given in Theorem 1
np.linalg.norm(model.Wout.weight.detach(),ord=2) * \
np.linalg.norm(model.mon.linear_module.U.weight.detach(),ord=2) / m

3.83663330078125

<h2> Lipschitz constant of Single Conv monDEQ model

In [6]:
# Instantiate and load single conv monDEQ model
m = 20
model = train.SingleConvNet(sp.MONPeacemanRachford,
                        in_dim=28,
                        in_channels=1,
                        out_channels=128,
                        alpha=1.0,
                        max_iter=300,
                        tol=1e-8,
                        m=m)
model.load_state_dict(torch.load("models/mon_1c_m20.0.pt",
                                 map_location=torch.device('cpu')))
model.eval()

SingleConvNet(
  (mon): MONPeacemanRachford(
    (linear_module): MONSingleConv(
      (U): Conv2d(1, 128, kernel_size=(3, 3), stride=(1, 1))
      (A): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), bias=False)
      (B): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), bias=False)
    )
    (nonlin_module): MONBorderReLU()
  )
  (Wout): Linear(in_features=6272, out_features=10, bias=True)
)

In [7]:
# Compute Lipschitz Constant of monDEQ as given in Theorem 1
Ufft = mon.init_fft_conv(model.mon.linear_module.U.weight, (28, 28))
np.linalg.norm(model.Wout.weight.detach(),ord=2) * \
torch.max(torch.svd(Ufft)[1]) / m

tensor(48.5502, grad_fn=<DivBackward0>)

<h2> Lipschitz constant of Multi Conv monDEQ model

In [11]:
# Instantiate and load multi conv monDEQ model
m = 20
model = train.MultiConvNet(sp.MONPeacemanRachford,
                        in_dim=28,
                        in_channels=1,
                        conv_sizes=(32, 64, 128),
                        alpha=2.0,
                        max_iter=300,
                        tol=1e-8,
                        m=m)

model.load_state_dict(torch.load("models/mon_mc_m20.0.pt",
                                 map_location=torch.device('cpu')))
model.eval()

MultiConvNet(
  (mon): MONPeacemanRachford(
    (linear_module): MONMultiConv(
      (U): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
      (A0): ModuleList(
        (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
        (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
        (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), bias=False)
      )
      (B0): ModuleList(
        (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
        (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
        (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), bias=False)
      )
      (A_n0): ModuleList(
        (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (1): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), bias=False)
      )
      (g): ParameterList(
          (0): Parameter containing: [torch.FloatTensor of size ]
          (1): Parameter containing: [torch.FloatTensor of size ]


In [12]:
# Compute Lipschitz Constant of monDEQ as given in Theorem 1
Ufft = mon.init_fft_conv(model.mon.linear_module.U.weight, (28, 28))
np.linalg.norm(model.Wout.weight.detach(),ord=2) * \
torch.max(torch.svd(Ufft)[1]) / m

tensor(198.2189, grad_fn=<DivBackward0>)